In [1]:
# run in docker:
# sudo docker run -p 8890:8888 -v /home/checkout/dir/pic2kcal:/tf/notebooks --runtime=nvidia -it --rm tensorflow/tensorflow:1.12.0-gpu-py3 jupyter notebook --allow-root --notebook-dir=/tf/notebooks
# crashes in TF1.4!
# does not work with Arch Linux TF installation!

In [2]:
%cd ..

/tf/notebooks


In [3]:
!pip install tensorflow-hub
!pip install sentencepiece
!pip install tf-sentencepiece==0.1.82.1
!pip install tqdm

You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!apt install -y jq

Reading package lists... Done
Building dependency tree       
Reading state information... Done
jq is already the newest version (1.5+dfsg-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

# Set up graph.
g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  en_de_embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-xling/en-de/1")
  embedded_text = en_de_embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Initialize session.
session = tf.Session(graph=g)
session.run(init_op)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0229 18:40:45.340061 139839549466368 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [6]:
from pathlib import Path
import json
import re
import random
import heapq
from operator import itemgetter
from tqdm.notebook import trange, tqdm
from math import ceil

data_dir = Path("data")

In [14]:
![ -f data/fddb_data.jsonl ] || jq '.[]' data/fddb_data.json > data/fddb_data.jsonl

In [15]:
![ -f data/fddb_data_withimg.json ] || jq 'select(([.Bilder[]|select(.title != "Noch kein Foto vorhanden.")]|length) > 0)' data/fddb_data.jsonl |\
    jq -s . > data/fddb_data_withimg.json

In [16]:
![ -f data/recipes/ingredients.json ] || jq '.ingredients[]|select(.ingredient)|.ingredient' data/recipes/processed_data.jsonl | jq -s unique > data/recipes/ingredients.json

In [17]:
with open(str(data_dir / "fddb_data_withimg.json"), encoding='utf-8') as f:
    fddb = json.load(f)
    # todo: make unique here
    _out_names = [e["name"] for e in fddb]

# jq '.ingredients[]|select(.ingredient)|.ingredient' processed_data.jsonl | jq -s unique > ingredients.json
with open(str(data_dir / "recipes/ingredients.json"), encoding='utf-8') as f:
    _in_names = json.load(f)


In [18]:
def normalize_ingredient(ing: str):
    # ing = re.sub(r"\(([^)])\)", "\g<1>", ing)  # remove stuff in parens
    ing = re.sub(r"\([^)]+\)", "", ing)  # remove stuff in parens
    ing = re.sub(r"(\d+,)?\d+ k?g\b", "", ing)  # remove xyz gram
    ing = re.sub(r",.*", "", ing) # TODO: DONT DO THIS! SIGNIFICANT INFORMATION LOSS
    ing = re.sub(r"\bzum .*", "", ing)
    ing = re.sub(r"\bfür .*", "", ing)
    ing = re.sub(r"\boder\b.*", "", ing)
    ing = ing.strip()
    return ing

from extract_ingredients.util import normalize_out_ingredient
    

In [19]:
in_names = list({normalize_ingredient(ing) for ing in _in_names})
print(len(_in_names))
in_names.sort()
len(in_names)

374670


26324

In [20]:
print(len(_out_names))
out_names = list({ning for ing in _out_names for ning in normalize_out_ingredient(ing)})
out_names.sort()
len(out_names)

122977


147431

In [21]:
#def Matcher():
#    def __init__(self, data_left, data_right, preproc_left, preproc_right):
#        pre_left = list(map(preproc_left, data_left))
#        self.left_vecs = list(zip(data_left, pre_left, get_sentence_vectors(pre_left)))
#        
#        pre_right = list(map(preproc_right, data_right))
#        self.right_vecs = list(zip(data_right, pre_right, get_sentence_vectors(pre_right)))

In [22]:
def make_chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

In [23]:
def get_sentence_vectors(texts):
    bs = 10000
    ccount = len(texts)//bs
    chunks = make_chunks(texts, bs)
    if ccount >= 3:
        chunks = tqdm(chunks, total=ccount)
    for chunk in chunks:
        yield from session.run(embedded_text, feed_dict={text_input: chunk})

In [24]:
def get_match(search: np.array, out_vecs, limit=30):
    it = ((v[0], np.dot(v[1], search)) for v in out_vecs)
    res_list = heapq.nlargest(limit, it, key=itemgetter(1))
    return res_list

In [25]:
out_vecs = list(zip(out_names, get_sentence_vectors(out_names)))

In [26]:
out_vecs[0][1].shape

(512,)

In [27]:
in_names[0], out_names[0]

('8 Kräuter', '"DIE FEINE" Geflügel-Fleischwurst')

In [28]:
in_vecs = list(zip(in_names, get_sentence_vectors(in_names)))

In [29]:
for ingredient, vec in random.sample(in_vecs, 100):
    res_list = get_match(vec, out_vecs)
    print("{} -> {}".format(ingredient, res_list[0]))

Obst noch dazu nach Geschmack -> ('Fruit to go', 0.8392326)
Gänsebrüste -> ('Gänsebrust', 0.8660896)
Ziegenfleisch -> ('Ziegenfeta', 0.9113623)
Champignons mit Flüssigkeit -> ('Champignons eingelegt', 0.8540534)
Tomate tetrapack -> ('Gourmet Tomate', 0.78558564)
Ananas - Schnitz -> ('Ananas - Scheiben', 0.89500946)
BBQ-Sauce nach Wahl -> ('BBQ-Sauce', 0.76371264)
Palmfett z. B. Palmin -> ('Kokosfett', 0.7291008)
Schnittlauch bei Bedarf -> ('Schnittlauch', 0.8137975)
Rindersteak à 200 - -> ('Rinderminutensteak', 0.7443519)
Rosinen in Rim -> ('Rosinen', 0.78593117)
Innereien vom Zicklein -> ('Kleine Scheibchen', 0.78706014)
Lebensmittelfarbe und Smarties -> ('Back und Speisefarben', 0.7150064)
Rückenspeck vom Schwein -> ('Rückensteak vom Schwein', 0.90191245)
Geschenkeband -> ('Stücke', 0.80141395)
Chilischote bzw. Peperoni -> ('Peperoni, scharf', 0.8561614)
Vanillezucker Bourbon -> ('Bourbon Vanillezucker', 0.9733576)
Marzipan - Kücken und Fondant-Eier -> ('Marzipan-Ei, mit Schokolade ü

In [30]:
z = [inv for inv in in_vecs if inv[0] == 'Kartoffeln'][0]
get_match(z[1], out_vecs)

[('Kartoffeln', 1.0),
 ('Kartoffeln ', 1.0),
 ('Kartoffeln ganz', 0.89777976),
 ('Kartoffeln gekocht', 0.89632434),
 ('Kartoffelstock', 0.8959535),
 ('Kartoffeln, gegart', 0.89144516),
 ('Kartoffeln roh', 0.8905239),
 ('Kartoffeln gebacken', 0.89049083),
 ('Kartoffelscheiben', 0.89016193),
 ('Kartoffeln, ganz', 0.88885343),
 ('Kartoffeln suppe', 0.8838953),
 ('Ganze Kartoffeln', 0.88332695),
 ('Ganze Kartoffeln ', 0.88332695),
 ('Brat Kartoffeln ', 0.8827737),
 ('Kartoffel', 0.880857),
 ('Kartoffelnudeln', 0.8792579),
 ('Kartoffelsticks', 0.8778616),
 ('Kartoffelsnack', 0.8732352),
 ('Gekochte Kartoffeln', 0.8718474),
 ('Kartoffelpree ', 0.8716015),
 ('Kartoffeln, roh', 0.8711822),
 ('geschälte Kartoffeln', 0.8708629),
 ('Kartoffel Sticks', 0.8684819),
 ('Kartoffeln mehlig', 0.86703193),
 ('Kartoffelklöße', 0.86207986),
 ('Kartoffelpürree', 0.8606976),
 ('Kartoffelpüree', 0.86053073),
 ('Kartoffelpüree ', 0.8605306),
 ('Kartoffel-Gemüse', 0.86040795),
 ('Kartoffelpürre ', 0.8589697)]

In [31]:
z = list(get_sentence_vectors(["Schalotten"]))[0]
get_match(z, out_vecs, limit=30)

[('Schalotten', 0.99999994),
 ('Sülze', 0.8927401),
 ('Ringlotten', 0.87886477),
 ('Schaschlik', 0.87672347),
 ('Selters', 0.87629664),
 ('Selchspeck', 0.87524664),
 ('Leberkse', 0.87103057),
 ('Hustelinchen', 0.8681994),
 ('Schwartenmagen', 0.86798435),
 ('Karlchen', 0.8628453),
 ('Maiskölbchen', 0.8627955),
 ('Grützwust', 0.85984874),
 ('Mürbchen ', 0.8595171),
 ('Maisecke', 0.85870075),
 ('Kersen', 0.8584007),
 ('Käsknacker', 0.85838103),
 ('Cemen', 0.85761976),
 ('Lattich', 0.8575938),
 ('Hhnerfilet', 0.8569063),
 ('Ketschup', 0.8567475),
 ('Korneck', 0.85655665),
 ('Körnereck', 0.8554502),
 ('Krönchen', 0.8546831),
 ('Knoferl', 0.85430384),
 ('Landschinken', 0.854226),
 ('Landschinken ', 0.854226),
 ('Wicküler ', 0.85283446),
 ('Roggenbrtchen ', 0.8517977),
 ('Salchichn', 0.8514552),
 ('Rohschinken', 0.8509237)]

In [32]:
z = list(get_sentence_vectors([normalize_ingredient("Salz")]))[0]
get_match(z, out_vecs, limit=30)

[('Salz', 0.99999994),
 ('Salzgurken', 0.9089097),
 ('Gut Salzig', 0.9034263),
 ('Salzbrezeln', 0.8880923),
 ('Salzgurken, salzig', 0.8686831),
 ('Salzmandeln', 0.8676863),
 ('Salzgurke', 0.86386824),
 ('Salzbrezeln, Salzig', 0.8605131),
 ('Salzbrezel', 0.8582727),
 ('Salz-Mandeln', 0.85328484),
 ('salzbrezel', 0.8416535),
 ('Salz, Jodsalz', 0.8251067),
 ('Salz-Diamanten', 0.8203409),
 ('Salzbutter', 0.81924367),
 ('Tafelsalz', 0.81821257),
 ('Salzdillgurken', 0.8158476),
 ('Jod Salz', 0.81424797),
 ('Salz flutes, Salz', 0.81192493),
 ('Meersalz', 0.811814),
 ('Blutdruck Salz', 0.8090306),
 ('Salz-Stangen', 0.8021811),
 ('smorbar gesalzen', 0.79520434),
 ('Chips Salz', 0.79345155),
 ('Salzstangerl', 0.78856623),
 ('Saltufo', 0.78672373),
 ('Salzstange ', 0.7866405),
 ('Salz Kräcker', 0.780357),
 ('Corn, salz', 0.7801466),
 ('Salz Stangen', 0.7790837),
 ('Koblauch Salz', 0.77868164)]

In [33]:
z = list(get_sentence_vectors([normalize_ingredient("Möhre")]))[0]
get_match(z, out_vecs, limit=30)

[('Möhre', 1.0),
 ('Mürbchen ', 0.8661137),
 ('Maisecke', 0.8569766),
 ('Mairübchen', 0.84997964),
 ('Cemen', 0.84976923),
 ('Hirsegrieß', 0.8468807),
 ('Hirsebllchen', 0.8456995),
 ('Korneck', 0.8455518),
 ('Schmelzkse', 0.83843553),
 ('Hirsebrei', 0.8378885),
 ('Hirse', 0.8378306),
 ('Möhrchen', 0.83730924),
 ('Kareespeck', 0.8371331),
 ('Grützwust', 0.83625674),
 ('Hirse Griess', 0.8357909),
 ('Bergpfirsich', 0.8353781),
 ('Kümmelpärle', 0.8350318),
 ('Schalotten', 0.83413565),
 ('Kräcker', 0.8336719),
 ('Gutfried', 0.8334216),
 ('Körnereck', 0.83339566),
 ('Tafelkren', 0.8322476),
 ('Süssmais', 0.8322011),
 ('Karreespeck', 0.8319967),
 ('Karreespeck ', 0.8319967),
 ('Edelschinlen', 0.83168244),
 ('Rotbäckchen', 0.83162296),
 ('Schokostrudel', 0.8315231),
 ('Schokolinsen', 0.83025897),
 ('Krönchen', 0.8302106)]

In [34]:
z = list(get_sentence_vectors([normalize_ingredient("Zitronen")]))[0]
get_match(z, out_vecs, limit=30)

[('Zitronen Saft', 0.93052155),
 ('Zitronenrolle', 0.92803085),
 ('Zitronenwasser', 0.92116797),
 ('Zitronenschale', 0.91545546),
 ('Zitronencreme', 0.9146689),
 ('Zitronensaft', 0.914645),
 ('Zitronensaft ', 0.91464496),
 ('Zitronenschaum', 0.9134511),
 ('Zitronen-Pesto', 0.9065609),
 ('Zitronen limonade', 0.9027357),
 ('Zitronen T', 0.8955948),
 ('Zitronenmelisse', 0.89085466),
 ('Lemon', 0.89002806),
 ('Zitrone', 0.886102),
 ('Zitronenlimonde', 0.8857571),
 ('Zitronenlimonade', 0.8841474),
 ('Zitronen-Häppchen', 0.8795186),
 ('Zitronensauce', 0.874436),
 ('Zitronen Aroma', 0.8718133),
 ('Zitronen Limonade', 0.87002224),
 ('Zitronen Limonade ', 0.87002224),
 ('Zitronet', 0.868004),
 ('Zitronen T, Zitrone', 0.8661582),
 ('Zitronenzucker', 0.86572045),
 ('Zitronenwasser, Zitrone', 0.86054575),
 ('Zitronenkuchen', 0.8587699),
 ('Zitronen Kuchen', 0.85712105),
 ('Zitronenschalen', 0.8568181),
 ('Zitronenschalen ', 0.8568181),
 ('Zitronen-Rolle', 0.8536694)]

In [35]:
z = list(get_sentence_vectors([normalize_ingredient("Pfeffer")]))[0]
get_match(z, out_vecs, limit=30)

[('Pfeffer', 1.0),
 ('Pfefferonen', 0.8929687),
 ('Pfefferminz', 0.88500786),
 ('Pfefferbreze', 0.8844123),
 ('Pfeffer Rolle', 0.88227504),
 ('Pfefferblock', 0.880451),
 ('Pfeffer-Schinken', 0.87850463),
 ('Pfefferminze', 0.8713012),
 ('Pfeffer-Braten', 0.8676797),
 ('Pfefferbeiser', 0.86668116),
 ('Pfeffer-Beißer', 0.86633503),
 ('Pfefferminztee', 0.86202484),
 ('Pfefferbeisser', 0.8613458),
 ('Pfeffersäckchen', 0.8607872),
 ('Pfefferbraten', 0.8570889),
 ('Bunter Pfeffer', 0.8531077),
 ('Pfeffer, weiß', 0.84696233),
 ('Pfefferkarree', 0.8463141),
 ('Pfeffer Braten', 0.84038985),
 ('Pfeffernüsse', 0.83831143),
 ('Pfeffer, schwarz', 0.8356359),
 ('Pfefferoni', 0.83562803),
 ('Pfefferbeißer', 0.8338611),
 ('Grüner Pfeffer', 0.83032626),
 ('Pfeffer Soße', 0.8280723),
 ('Pfefferonen, scharf', 0.82477903),
 ('Paprikaschote', 0.82357407),
 ('Pfefferschinken', 0.8235053),
 ('Käse, Pfeffer', 0.81792927),
 ('Salz und Pfeffer', 0.8160138)]

In [36]:
z = list(get_sentence_vectors([normalize_ingredient("Zucchini")]))[0]
get_match(z, out_vecs, limit=30)

[('Zucchini', 1.0),
 ('Zucchini ', 1.0),
 ('Zuchini', 0.96570617),
 ('Zucchini-Puffer', 0.95276517),
 ('Zucchini-Gratin', 0.94445246),
 ('Zucchini Pfanne', 0.9339043),
 ('Zucchini Quiche', 0.9273448),
 ('Zucchini, gegart', 0.91964746),
 ('Zucchini-Pfanne', 0.9194037),
 ('Zucchini-Reistopf', 0.91909873),
 ('Zucchini-Fritata', 0.9189852),
 ('Zucchinipuffer', 0.915961),
 ('Zucchini gebraten', 0.90767103),
 ('Gebackene Zucchini', 0.90149117),
 ('Zucchini Püree', 0.89597404),
 ('Currychini', 0.8911982),
 ('Zucchini-puffer', 0.8880278),
 ('Zucchinibrot', 0.88706595),
 ('Zucchini Auflauf', 0.8870287),
 ('Zucchini Cannelloni', 0.88696444),
 ('Papucchini', 0.88294375),
 ('Zucchini-Cheddar-Puffer', 0.8814348),
 ('Zucchini Mischung', 0.8783588),
 ('Crocchini', 0.87423617),
 ('Zuchini gegrillt', 0.8742034),
 ('Zucchini-Paprika-Ragu', 0.8666969),
 ('Gegrillte Zucchini', 0.86394775),
 ('Zucchini Brownie', 0.8617482),
 ('Zucchini-Gemüse-Pfanne', 0.8587594),
 ('Zucchini-Spaghetti', 0.857448)]

In [37]:
z = list(get_sentence_vectors([normalize_ingredient("Paprikaschote")]))[0]
get_match(z, out_vecs, limit=30)

[('Paprikaschote', 1.0),
 ('Paprikaschoten', 0.98521686),
 ('Paprikagulasch', 0.9507465),
 ('Paprikasülze', 0.9479989),
 ('Paprika-Gulasch', 0.9312786),
 ('Paprikapastete', 0.9309405),
 ('Paprikastreifen', 0.9304861),
 ('Gefüllte Paprikaschote', 0.9249152),
 ('Paprikapulver', 0.9245887),
 ('Paprikacreme', 0.92321074),
 ('Paprikagemüse', 0.92314374),
 ('Paprikaknacker', 0.91852474),
 ('Paprikasalat', 0.91250634),
 ('Paprika Edelsüss', 0.9123242),
 ('Rote Paprikaschoten', 0.9121383),
 ('Paprikamark', 0.91010225),
 ('Paprikareis', 0.9083037),
 ('Paprika-Mandel-Dip', 0.90779126),
 ('Paprika-Bruscetta', 0.906524),
 ('Paprikasoße', 0.9050809),
 ('Paprika-Lyoner', 0.9030753),
 ('Eingelegte Paprika', 0.90060955),
 ('Paprikasticks', 0.8994635),
 ('Paprika-Chili', 0.89918077),
 ('Paprikasuppe', 0.89701134),
 ('Chilischote', 0.89657897),
 ('Paprika Quark', 0.89454365),
 ('Geräucherte Paprika', 0.8937136),
 ('Paprika Snackis', 0.8935468),
 ('Gefüllte Paprika', 0.8931304)]

In [38]:
z = list(get_sentence_vectors([normalize_ingredient("Zucker")]))[0]
get_match(z, out_vecs, limit=30)

[('Zucker', 0.99999994),
 ('Gelier Zucker', 0.9193795),
 ('Gelier Zucker ', 0.91937935),
 ('Zuckerbrezel', 0.8992935),
 ('Fein Zucker', 0.88426435),
 ('Südzucker', 0.8819196),
 ('Zucker-Streusel', 0.8814236),
 ('Fruchtzucker', 0.8783986),
 ('Milchzucker', 0.8741473),
 ('Palmzucker', 0.87408715),
 ('Brauner Zucker', 0.8705966),
 ('Traubenzucker', 0.8682995),
 ('Traubenzucker ', 0.8682995),
 ('Gelierzucker', 0.86561555),
 ('Zuckermais', 0.86528414),
 ('Zuckerkarotten', 0.86428124),
 ('Feiner Zucker', 0.8640773),
 ('Zuckerschoten', 0.86231935),
 ('Backzucker', 0.86181164),
 ('Feinzucker', 0.86068845),
 ('Süßstoff', 0.8601293),
 ('Rohrzucker', 0.8591718),
 ('Zitronenzucker', 0.85892457),
 ('Süßstoff, Zucker', 0.8552714),
 ('Zucker Sticks', 0.8548074),
 ('Puderzucker ', 0.8527151),
 ('Puderzucker', 0.852715),
 ('Zuckerstange', 0.8474886),
 ('Kandiszucker', 0.8458615),
 ('Zuckerfrei Bonbon', 0.8427985)]

In [39]:
z = list(get_sentence_vectors([normalize_ingredient("Knoblauch")]))[0]
get_match(z, out_vecs, limit=30)

[('Knoblauch', 0.99999994),
 ('Knoblauchgurken', 0.9575032),
 ('Knoblauchgurken ', 0.9575032),
 ('Knoblauchzehe', 0.9545717),
 ('Knoblauchspeck', 0.95325136),
 ('Knoblauchpüree', 0.94609964),
 ('Knoblauchschmalz', 0.94249153),
 ('Knoblauchecken', 0.93821514),
 ('Knoblauchsauce', 0.92497647),
 ('Knoblauchcreme', 0.91737366),
 ('Knoblauchcreme ', 0.91737366),
 ('Knoblauchbraten', 0.9143625),
 ('Knoblauch, getrocknet', 0.909333),
 ('Knoblauchbutter', 0.908661),
 ('Knoblauchsoße', 0.9085437),
 ('Knoblauch Braten', 0.90431464),
 ('Knoblauch  sauce', 0.90300894),
 ('Knoblauch sauce', 0.90300894),
 ('Knoblauch-Baguette', 0.90248096),
 ('Knoblauchkick', 0.9013521),
 ('Knoblauchsalami', 0.90006006),
 ('Knoblauch-Creme', 0.897392),
 ('Knoblauchsuppe', 0.8959119),
 ('Knoblauch Senf', 0.8951769),
 ('Knoblauch Pulver', 0.891781),
 ('Knoblauchbeißer', 0.8914834),
 ('Knoblauch, frisch', 0.8905188),
 ('Knoblauchbaguette', 0.88851404),
 ('Knoblauch Dressing', 0.88626647),
 ('Knoblauchecken, Knoblauch',

In [41]:
# i thought multiprocessing is impossible in jupyter notebook?
# better not question it
from multiprocessing import Pool, cpu_count

def jsonable(e):
    return [(a,float(b)) for a, b in e]

def get_match_w(t):
    ing, vec = t
    return ing, get_match(vec, out_vecs)

all_ings = {}
with Pool(cpu_count()) as pool:
    for ing, match in pool.imap(get_match_w, tqdm(in_vecs), chunksize=100):
        all_ings[ing] = jsonable(match)

In [42]:

all_ings_orig = {ing: all_ings[normalize_ingredient(ing)] for ing in _in_names}

In [43]:
with open("data/recipe-ingredient-to-fooddb.json", "w") as f:
    json.dump(all_ings_orig, f)